# Final Model Training

Here, we will train the final model and export it so it can be used in the web application.

Many of the same steps are taken in the other notebooks, so we will not focus too much on describing the steps that are being taken.

In [1]:
# so we have access to the Google Drive filesystem
#from google.colab import drive
#drive.mount('/content/drive')

In [1]:
# necessary imports
import os
import pandas as pd
import numpy as np

# so we can access local modules within Colab
#os.chdir('/content/drive/My Drive/auto-age-detector-model')

# feature selection defined functions
from feature_selection import tree_based_feature_selection

# model creation
from models import create_model

# for feature scaling
from sklearn.preprocessing import StandardScaler

Using TensorFlow backend.


In [2]:
df_train = pd.read_csv('C:\\Users\\gotty\\Desktop\\project\\models\\sample.csv').drop(columns=['Unnamed: 0','path'])
# drop any null values we may have forgotten
df_train = df_train.dropna(how='any',axis=0)
X_train = df_train.drop(columns=['age'],axis=1)
y_train = df_train['age']

In [3]:
X_train,data_transformer = tree_based_feature_selection(X_train,y_train,
                                                        n_estimators=75)

In [4]:
replaced = {'teens':0,'twenties':1,'thirties':2,'fourties':3,'fifties':4,
            'sixties':5,'seventies':6,'eighties':7,'nineties':8}

# https://stackoverflow.com/questions/29831489/convert-array-of-indices-to-1-hot-encoded-numpy-array

# need to put one hot encoded in keras model
y_train_ohe = y_train.replace(replaced).astype(int)
from numpy import array
y_train_ohe = array(y_train_ohe)
y_train_ohe = np.eye(np.max(y_train_ohe)+1)[y_train_ohe]

In [5]:
# standard scaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)

In [6]:
# ensure reproducible results
from numpy.random import seed
seed(1)

In [7]:
X_train.shape

(31, 69)

In [8]:
# get the model
model = create_model(dropout=0.1,learning_rate=1e-3,output_classes=4)

model.fit(X_train,y_train_ohe,batch_size=32,validation_split=0.15,
          epochs=10)


Train on 26 samples, validate on 5 samples
Epoch 1/10


InternalError: 2 root error(s) found.
  (0) Internal: Blas GEMM launch failed : a.shape=(26, 69), b.shape=(69, 128), m=26, n=128, k=69
	 [[{{node dense_1/MatMul}}]]
	 [[Mean/_117]]
  (1) Internal: Blas GEMM launch failed : a.shape=(26, 69), b.shape=(69, 128), m=26, n=128, k=69
	 [[{{node dense_1/MatMul}}]]
0 successful operations.
0 derived errors ignored.

We save the model.

In [0]:
model.save('model1.h5')
from google.colab import files
files.download(os.path.join(os.getcwd(),'model1.h5'))

We also save the feature selector and scaler for consistency.

In [0]:
import pickle
with open('feature_selector.pkl','wb') as feat_sel:
  pickle.dump(data_transformer,feat_sel)

In [0]:
with open('standard_scaler.pkl','wb') as standard_scaler:
  pickle.dump(scaler,standard_scaler)

In [36]:
test = np.array([1,3,4,3])
isinstance(test,np.ndarray)

True